<a href="https://colab.research.google.com/github/patelami3431/Web-Crawler/blob/master/HLT_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import urllib
import requests
import bs4
from bs4 import BeautifulSoup
import pandas as pd
import re

In [0]:
import urllib
from urllib import request
import requests
import bs4
from bs4 import BeautifulSoup
import pandas as pd
import re

url = "https://en.wikipedia.org/wiki/Fruitarianism"
html = request.urlopen(url).read()
soup = BeautifulSoup(html)

# kill all script and style elements
for script in soup(['style', 'script', '[document]', 'head', 'title']):
    script.extract()    # rip it out

# get text
text = soup.get_text()

# break into lines and remove leading and trailing space on each
lines = (line.strip() for line in text.splitlines())
# break multi-headlines into a line each
chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
# drop blank lines
text = ' '.join(chunk for chunk in chunks if chunk)
with open('wiki.txt', 'w+') as f:
  f.write(text)
f.close()

In [0]:
from nltk.tokenize import sent_tokenize
sents = sent_tokenize(text)
for sent in sents:
    if 'Retrieved ' not in sent:
        print(sent)

In [12]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

## reading text from each file, tokenizing sentence and write tokenized sentence to file.

In [0]:
from nltk.tokenize import sent_tokenize
from nltk.corpus import stopwords

def sent_tokenizer(filename):
  file = open(filename,"r")
  content = file.read()
  content = content.lower()
  punctuations = '''!()-[]{};:'"\,<>/?@#$%^&*_~'''
  for x in content: 
        if x in punctuations: 
            content = content.replace(x, "") 
  sent = sent_tokenize(content)
  with open('tokenzied_'+filename, 'w+') as f:
    for s in sent:
      f.write(s+'\n')
  f.close()
sent_tokenizer('wiki.txt')